<a href="https://colab.research.google.com/github/Corona-Locator-Nederland/corona-locator-nederland/blob/main/Corona_locator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from urllib.request import urlopen
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import json
import datetime
import requests
import json
import math
import os
import seaborn as sns
import glob
from dateutil.parser import parse as parsedate

from dotenv import load_dotenv, find_dotenv
try:
  load_dotenv(find_dotenv())
except IOError:
  load_dotenv(find_dotenv(filename='dot.env'))


Download de RIVM data als die nieuwer is dan wat we al hebben (gecached want de download van RIVM is *zeer* traag)

In [3]:
os.makedirs('downloads', exist_ok = True)

rivm = requests.head('https://data.rivm.nl/covid-19/COVID-19_casus_landelijk.csv')
latest = os.path.join('downloads', parsedate(rivm.headers['last-modified']).strftime('%Y-%m-%d@%H-%M.csv'))
if not os.path.exists(latest):
  print('downloading', latest)
  for f in glob.glob('downloads/*.csv'):
    if f != latest:
      os.remove(f)
  urlretrieve('https://data.rivm.nl/covid-19/COVID-19_casus_landelijk.csv', latest)
else:
  print(latest, 'exists')
covid = pd.read_csv(latest, sep=';', header=0 )
covid.head()

downloads/2021-01-05@14-15.csv exists


,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Hospital_admission,Deceased,Week_of_death,Municipal_health_service
0,2021-01-05 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,No,NaN,GGD Amsterdam
1,2021-01-05 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2021-01-05 10:00:00,2020-01-02,DOO,70-79,Female,Noord-Holland,No,No,NaN,GGD Gooi en Vechtstreek
3,2021-01-05 10:00:00,2020-01-16,DOO,0-9,Female,Zuid-Holland,No,No,NaN,GGD Rotterdam-Rijnmond
4,2021-01-05 10:00:00,2020-01-20,DOO,50-59,Female,Gelderland,No,No,NaN,GGD Gelderland-Zuid


Download de bevolkings cijfers van CBS, uitgesplitst op de leeftijds categorien in de dataset van het RIVM

In [4]:
# https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/snelstartgids-odata-v4
def get_odata(target_url):
  data = pd.DataFrame()
  while target_url:
    r = requests.get(target_url).json()
    data = data.append(pd.DataFrame(r['value']))
        
    if '@odata.nextLink' in r:
      target_url = r['@odata.nextLink']
    else:
      target_url = None          
  return data

def roundup(x):
  return int(math.ceil(x / 10.0)) * 10
def rounddown(x):
  return int(math.floor(x / 10.0)) * 10

cbs = 'https://opendata.cbs.nl/ODataApi/OData/83482NED'

leeftijden = get_odata(cbs + "/Leeftijd?$select=Key, Title&$filter=CategoryGroupID eq 3")
leeftijden.set_index('Key', inplace=True)
# zet de Title om naar begin-eind paar
leeftijden_range = leeftijden['Title'].replace(r'^(\d+) tot (\d+) jaar$', r'\1-\2', regex=True).replace(r'^(\d+) jaar of ouder$', r'\1-1000', regex=True)
# splits die paren in van-tot
leeftijden_range = leeftijden_range.str.split('-', expand=True).astype(int)
# rond the "van" naar beneden op tientallen, "tot" naar boven op tientallen, en knip af naar "90+" om de ranges uit de covid tabel te matchen
leeftijden_range[0] = leeftijden_range[0].apply(lambda x: rounddown(x)).apply(lambda x: str(x) if x < 90 else '90')
leeftijden_range[1] = (leeftijden_range[1].apply(lambda x: roundup(x)) - 1).apply(lambda x: f'-{x}' if x < 90 else '+')
# en plak ze aan elkaar
leeftijden['Range'] = leeftijden_range[0] + leeftijden_range[1]
del leeftijden['Title']

def query(f):
  if f == 'Leeftijd':
    # alle leeftijds categerien zoals hierboven opgehaald
    return '(' + ' or '.join([f"{f} eq '{k}'" for k in leeftijden.index.values]) + ')'
  if f in ['Geslacht', 'Migratieachtergrond', 'Generatie']:
    # pak hier de key die overeenkomt met "totaal"
    ids = get_odata(cbs + '/' + f)
    return f + " eq '" + ids[ids['Title'].str.contains("totaal", na=False, case=False)]['Key'].values[0] + "'"
  if f == 'Perioden':
    # voor perioden pak de laatste
    return f + " eq '" + get_odata(cbs + '/Perioden').iloc[[-1]]['Key'].values[0] + "'"
  raise ValueError(f)
# haal alle properties op waar op kan worden gefiltered en stel de query samen
filter = get_odata(cbs + '/DataProperties')
filter = ' and '.join([query(f) for f in filter[filter.Type != 'Topic']['Key'].values])

bevolking = get_odata(cbs + f"/TypedDataSet?$top=100&$filter={filter}&$select=Leeftijd, BevolkingOpDeEersteVanDeMaand_1")
# die _1 betekent waarschijnlijk dat het gedrag ooit gewijzigd is en er een nieuwe "versie" van die kolom is gepubliceerd
bevolking.rename(columns = {'BevolkingOpDeEersteVanDeMaand_1': 'BevolkingOpDeEersteVanDeMaand'}, inplace = True)
bevolking = bevolking.merge(leeftijden, left_on = 'Leeftijd', right_index = True)
# optellen om de leeftijds categorien bij elkaar te vegen om de "agegroups" uit "covid" te matchen 
bevolking = bevolking.groupby('Range')['BevolkingOpDeEersteVanDeMaand'].sum().to_frame()
bevolking['per 100k'] = 100000 / bevolking['BevolkingOpDeEersteVanDeMaand']
bevolking

,BevolkingOpDeEersteVanDeMaand,per 100k
Range,,
0-9,1759195,0.056844
10-19,1987617,0.050312
20-29,2240262,0.044638
30-39,2178284,0.045908
40-49,2169549,0.046093
50-59,2547659,0.039252
60-69,2139971,0.046730
70-79,1613006,0.061996
80-89,706728,0.141497


Bereken de stand van zaken van besmettingen / hospitalisaties / overlijden, per cohort in absolute aantallen en aantallen per 100k, met een kleur indicator voor de aantallen.

In [81]:
# vervang <50 en Unknown door Onbekend
covid['Cohort'] = covid['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
# aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in covid['Cohort'].unique() if cohort[0].isdigit()]
assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
covid['Gemiddelde leeftijd'] = covid['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

# verwijder tijd
covid['Date_file_date'] = pd.to_datetime(covid['Date_file'].replace(r' .*', '', regex=True))

covid['Date_statistics_date'] = pd.to_datetime(covid['Date_statistics'])

# weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
covid['Weken terug'] = np.floor((covid['Date_file_date'] - covid['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(np.int)

# voeg key, gem leeftijd, kleurnummer en totaal toe
Date_file = covid['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
cohorten = list(bevolking.index) + ['Onbekend']
def summarize(df, category, prefix):
  # doesn't matter what we count as long as it's guaranteed to be filled
  df = (df.copy(deep=True)
        .groupby(['Weken terug', 'Cohort'])['Date_file_date']
        .count()
        .unstack(fill_value=np.nan)
        .reset_index()
        .rename_axis(None, axis=1)
      ).merge(df.copy(deep=True) # voeg hier gemiddelde leeftijd toe, want die willen we op een ander niveau aggregeren voor 'df' overschreven word
        .groupby(['Weken terug'])['Gemiddelde leeftijd']
        .mean()
        .to_frame(), on='Weken terug'
      )

  # altijd 52 rijen
  df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df.copy(deep=True), how='left', on='Weken terug')

  # insert missing cohorts
  for col in cohorten:
    if not col in df:
      df[col] = np.nan

  # sommeer over de cohorten om een totaal te krijgen 
  df['Totaal'] = df[cohorten].sum(axis=1)

  # voeg periode en datum toe
  # periode afgeleid van weken-terug (= de index voor deze dataframe)
  df['Datum'] = Date_file
  df['Periode'] = (df
    .index.to_series()
    .apply(
      lambda x: (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m') + ' - ' + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
    )
  )

  # voeg 'Key' en 'Type' kolom toe
  df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
  df['Type'] = category

  # voeg de kleur kolommen toe
  for col in cohorten:
    df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

  # herschikken van de kolommen om totaal vooraan te krijgen
  colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
  return df[colorder + [col for col in df if col not in colorder]]

def per100k(df):
  df = df.copy(deep=True)
  for col in cohorten:
    if col in df:
      factor = bevolking.loc[col]['per 100k'] if col in bevolking.index else np.nan
      df[col] = df[col] * factor
  return df

subsets = [
  (covid, 'Positief getest', 'p'), # volledige count per cohort
  (covid[covid.Hospital_admission == 'Yes'], 'Ziekenhuisopname', 'h'), # count van cohort voor Hospital_admission == 'Yes'
  (covid[covid.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
]
rivm = None
for dataset, category, prefix in subsets:
  if rivm is None:
    rivm = summarize(dataset, category, prefix)
  else:
    rivm = pd.concat([rivm, summarize(dataset, category, prefix)])
  rivm = pd.concat([rivm, summarize(per100k(dataset), category + ' per 100.000', prefix + '100k')])

#cm = sns.light_palette("green", as_cmap=True)
#postest[[col for col in postest.columns if col[0] == 'r' and col[1] != 'c' and not col.startswith('r100')]].style.background_gradient(cmap=cm, axis=1)
rivm.fillna(0).head()


,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,...,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2021-01-05,30/12 - 05/01,44.804839,25005.0,Positief getest,434.0,2977.0,4453.0,...,668,1000,741,757,959,596,403,284,105,0
1,p001,1,2021-01-05,23/12 - 29/12,45.158832,56162.0,Positief getest,863.0,6586.0,9077.0,...,657,905,743,833,1000,617,403,268,87,0
2,p002,2,2021-01-05,16/12 - 22/12,44.199673,74535.0,Positief getest,1154.0,10125.0,11333.0,...,764,855,745,889,1000,606,362,234,81,0
3,p003,3,2021-01-05,09/12 - 15/12,42.723033,73011.0,Positief getest,1291.0,12084.0,10719.0,...,976,866,779,927,1000,593,354,220,76,0
4,p004,4,2021-01-05,02/12 - 08/12,41.774415,53799.0,Positief getest,982.0,9920.0,7850.0,...,1000,791,708,854,880,527,312,185,64,0


In [76]:
for cond in (pd.Series(np.repeat(True, len(df.index))), covid.Hospital_admission == 'Yes'):
    print(covid[cond].head())

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

Publiceer de berekende statistieken indien we op github draaien


In [7]:
# publish
if 'GITHUB_TOKEN' in os.environ:
  os.makedirs('artifacts', exist_ok = True)
  print('Publishing to', os.environ['GITHUB_REPOSITORY'])
  today = os.path.join('artifacts', datetime.date.today().strftime('%Y-%m-%d') + '.csv')
  latest = 'artifacts/covid.csv'
  rivm.fillna(0).to_csv(latest, index=False)

  import github3 as github
  gh = github.GitHub(token=os.environ['GITHUB_TOKEN'], session=github.session.GitHubSession(default_read_timeout=60))
  repo = gh.repository(*os.environ['GITHUB_REPOSITORY'].split('/'))
  release = repo.release_from_tag('covid')
  assets = { asset.name: asset for asset in release.assets() }

  # remove existing
  for asset in [today, latest]:
    if os.path.basename(asset) in assets:
      assets[os.path.basename(asset)].delete()
    with open(latest) as f:
      release.upload_asset(asset=f, name=os.path.basename(asset), content_type='text/csv')